# Homework 2

Due before class on Thursday, February 7

## Question 1

Write a function that returns the name of a company for a given ticker symbol. 

You should scrape the data from Yahoo! Finance. [Here's an example page](https://finance.yahoo.com/quote/WMT/profile) using Walmart. Start by making your function modify the URL so it works for other ticker symbols. Then, look at the HTML source to find the pattern that you must search for to extract the company name.

I've started the function for you here.

In [33]:
def get_name(ticker):
    import requests, re
    
    # put the URL here
    
    url = 'https://finance.yahoo.com/quote/{}/profile'.format(ticker)
    req = requests.get(url)
    html = req.text

    
    try:
        # use a regular expression to find the name
        name = re.search(r'><title>{} Profile \| (.*?) Stock'.format(ticker), html).group(1)
        return name
    
    except:
        # if the regular expression fails, do nothing
        pass

Test your function using these tickers: `AMZN`, `WMT`, `MMM`. Be sure it works as expected.

In [34]:
get_name('MMM')

'3M Company'

## Question 2
Write a function that uses the same web page of data to find the address of the company's headquarters.

For example, your function should work as follows:

    In  []: get_address('TSLA')
    Out []: '3500 Deer Creek Road, Palo Alto, CA, 94304'
    
Be sure to test your function with some other stock tickers.

Hint: You'll probably want to capture different parts of the address (street address, city, state, zip) separately and then join them together into a string before returning a value. 

In [24]:
def get_address(ticker):
    import requests, re
    
    try:
        url = 'https://finance.yahoo.com/quote/{}/profile'.format(ticker)
        req = requests.get(url)
        html = req.text
        street = re.search(r'address1":"(.+)\",\"fax"',html)
        if street is not None:
            street = re.search(r'address1":"(.+)","fax"',html).group(1)    
        else:
            street = re.search(r'address1":"(.+)","industry"',html).group(1)
        city = re.search(r'city":"(.+)","phone',html).group(1)
        state = re.search(r'state":"(\w+)","shareHolderRightsRisk',html).group(1)
        zip_code =  re.search(r'zip":"(\d+)","sector"', html).group(1)
        address = ', '.join([street,city,state,zip_code])
        return address
    
    except:
        pass

In [35]:
get_address('WMT')

'702 SW 8th Street, Bentonville, AR, 72716'

## Question 3

Now I want you to write a *geolocating* function that takes a ticker symbol and returns the latitude and longitude of the firm's headquarters. To do this, we will use Mapquest's API (**A**pplication **P**rogramming **I**nterface), which is a web-based system for providing raw data rather than HTML pages.

Before accessing the API you need to create an account which generates an API Key. The account is free and allows you to make a limited number of queries. Once you have created your key, you can query the API using a URL like this:

http://www.mapquestapi.com/geocoding/v1/address?key=API_KEY&location=1309+E+10th+Street%2C+Bloomington%2C+IN%2C+47401

(This link won't work until you replace `API_KEY` with your personal key.)

To do this query from python, use `requests` like we did in class:

In [36]:
import requests, re

API_KEY = 'bPuxRuQQPkHdDYLApq18OB3UesUo9r5O'
url = 'http://www.mapquestapi.com/geocoding/v1/address'
params = {'key': API_KEY, 'location': '1309 E 10th Street, Bloomington, IN, 47401'}
req = requests.get(url, params=params)

The response from the server is data from the database stored in JSON format. You can process it as JSON, or you can ignore that it's JSON and just use a regular expression to find the pattern you're looking for.

As an example, your code should return a tuple *exactly* like this:

    In  [2]: geolocate('TSLA')
    Out [2]: (37.394738, -122.149112, '3500 Deer Creek Road, Palo Alto, CA, 94304')
    
That is, it returns a tuple with the latitude, longitude, and address. Note that the lat/lng coordinates are `float` values.

The function you write for this question should start by calling the function you wrote in Q2 to get the address. You can then use this with the API to get the location

In [37]:
def geolocate(ticker):
    import requests, re
    
    try:
        address = get_address(ticker)
        API_KEY = 'bPuxRuQQPkHdDYLApq18OB3UesUo9r5O'
        url_2 = 'http://www.mapquestapi.com/geocoding/v1/address'
        params = {'key': API_KEY, 'location': address}
        req = requests.get(url_2, params=params)
        lat = float(re.search(r'\"latLng":{"lat":([?-]?\d*\.\d*),"lng":([?-]?\d*\.\d*)}', req.text).group(1))
        lng = float(re.search(r'\"latLng":{"lat":([?-]?\d*\.\d*),"lng":([?-]?\d*\.\d*)}', req.text).group(2))
        geo_location = tuple([lat,lng,address])
        return geo_location
    
    except:
        pass

In [39]:
geolocate('AMZN')

(47.622333, -122.33717, '410 Terry Avenue North, Seattle, WA, 98109')

## Question 4

Now let's say we want to calculate the distance between two corporate headquarters.

Given two pairs of latitude and longitude, $(\varphi_1, \lambda_1)$ and $(\varphi_2, \lambda_2)$, we can calculate the distance between them using the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula#The_haversine_formula):

$${\displaystyle d=2r\arcsin \left({\sqrt {\operatorname {hav} (\varphi _{2}-\varphi _{1})+\cos(\varphi _{1})\cos(\varphi _{2})\operatorname {hav} (\lambda _{2}-\lambda _{1})}}\right)}$$

where $d$ is the distance, $\operatorname{hav}$ is the Haversine function, $$\operatorname {hav} (\theta )=\sin ^{2}\left({\frac {\theta }{2}}\right),$$ and $r$ is the radius of the earth ([3958.7613 miles](https://en.wikipedia.org/wiki/Earth_radius#Mean_radius)). Note that this formula requires that the latitude and longitude are given in *radians*, not the degrees that Mapquest gives.

Use the template below to write a function that takes two ticker symbols and returns the distance between the headquarters of the two firms. Note that the python function for $\arcsin$ is `asin`.

Test your function with several pairs of ticker symbols.

In [29]:
def haversine(theta):
    '''Haversine function'''
    from math import sin
    return sin(theta/2)**2

def firm_dist(ticker1, ticker2):
    '''Returns the distance in miles between two firm HQs'''
    from math import radians, cos, sin, asin, sqrt
 
    
    try:
        # get lat/lng of the two tickers
        
        coord_1 = geolocate(ticker1)
        lat_1 = coord_1[0]
        lng_1 = coord_1[1]
    
        coord_2 = geolocate(ticker2)
        lat_2 = coord_2[0]
        lng_2 = coord_2[1]

        # change degrees to radians (hint: use "map")

        coord_rads = tuple(map(radians, [lat_1,lng_1,lat_2,lng_2]))
    
        # calculate and return the distance using the the formula given above

        distance = 2*3958.7613*asin(sqrt(haversine(coord_rads[2]-coord_rads[0])+cos(coord_rads[0])*
                                cos(coord_rads[2])*haversine(coord_rads[3]-coord_rads[1])))

        return(distance)
    
    except:
        pass

In [40]:
# test your function
firm_dist('TSLA','AMZN')


706.723812722618